In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os
from torch.utils import data
from wavenet import Wavenet
from transformData import x_mu_law_encode,y_mu_law_encode,mu_law_decode,onehot,cateToSignal
from readDataset2 import Dataset,Testset

In [2]:
sampleSize=16000#the length of the sample size
quantization_channels=256
sample_rate=16000
dilations=[2**i for i in range(9)]*7  #idea from wavenet, have more receptive field
residualDim=128 #
skipDim=512
shapeoftest = 190500
filterSize=3
songnum=10
savemusic='./vsCorpus/notextr{}.wav'
resumefile='./model/testac' # name of checkpoint
lossname='testacloss.txt' # name of loss file
continueTrain=False # whether use checkpoint
pad = np.sum(dilations) # padding for dilate convolutional layers
lossrecord=[]  #list for record loss
sampleCnt=0
#pad=0

    #            |----------------------------------------|     *residual*
    #            |                                        |
    #            |    |-- conv -- tanh --|                |
    # -> dilate -|----|                  * ----|-- 1x1 -- + -->	*input*
    #                 |-- conv -- sigm --|     |    ||
    #                                         1x1=residualDim
    #                                          |
    # ---------------------------------------> + ------------->	*skip=skipDim*
    image changed from https://github.com/vincentherrmann/pytorch-wavenet/blob/master/wavenet_model.py

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # use specific GPU

In [4]:
use_cuda = torch.cuda.is_available() # whether have available GPU
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#device = 'cpu'
#torch.set_default_tensor_type('torch.cuda.FloatTensor') #set_default_tensor_type as cuda tensor

In [5]:
params = {'batch_size':2, 'shuffle': True, 'num_workers': 2}
training_set = Dataset(np.arange(0, songnum), np.arange(0, songnum), 'ccmixter2/x/', 'ccmixter2/y/')
validation_set = Testset(np.arange(0, songnum), 'ccmixter2/x/')
loadtr = data.DataLoader(training_set, **params)  # pytorch dataloader, more faster than mine
loadval = data.DataLoader(validation_set,batch_size=1)

In [6]:
model = Wavenet(pad,skipDim,quantization_channels,residualDim,dilations).cuda()
criterion = nn.CrossEntropyLoss()
#in wavenet paper, they said crossentropyloss is far better than MSELoss
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-5)
#use adam to train
#optimizer = optim.SGD(model.parameters(), lr = 0.1, momentum=0.9, weight_decay=1e-5)
#scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
#scheduler = MultiStepLR(optimizer, milestones=[20,40], gamma=0.1)

In [7]:
start_epoch=0
if continueTrain:# if continueTrain, the program will find the checkpoints
    if os.path.isfile(resumefile):
        print("=> loading checkpoint '{}'".format(resumefile))
        checkpoint = torch.load(resumefile)
        start_epoch = checkpoint['epoch']
        #best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(resumefile, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(resumefile))

In [ ]:
def test(xtrain, iloader):  # testing data
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        for iloader, (xtrain, ytrain) in enumerate(loadval):
            listofpred = []
            for ind in range(pad, xtrain.shape[-1] - pad, sampleSize):
                output = model(xtrain[:, :, ind - pad:ind + sampleSize + pad].to(device))
                pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
                listofpred.append(pred)
            ans = mu_law_decode(np.concatenate(listofpred))
            sf.write(savemusic.format(iloader), ans, sample_rate)
            print('test stored done', np.round(time.time() - start_time))


def train(epoch):  # training data, the audio except for last 15 seconds
    model.train()
    for iloader, (xtrain, ytrain) in enumerate(loadtr):
        start_time = time.time()
        data, target = xtrain.to(device), ytrain.to(device)
        output = model(data)
        loss = criterion(output, target)
        lossrecord.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        global sampleCnt
        sampleCnt+=1
        print('Train Epoch: {} iloader:{} Loss:{:.6f}: , ({:.3f} sec/step)'.format(
            epoch, iloader, loss.item(), time.time() - start_time))
        if sampleCnt % 10000 == 0 and sampleCnt > 0:
            for param in optimizer.param_groups:
                param['lr'] *= 0.98

    if epoch % 100 == 0 and epoch > 0:
        with open("./lossRecord/" + lossname, "w") as f:
            for s in lossrecord:
                f.write(str(s) + "\n")
        print('write finish')
        state = {'epoch': epoch + 1,
                 'state_dict': model.state_dict(),
                 'optimizer': optimizer.state_dict()}
        torch.save(state, resumefile)
        test(xtrain, iloader)

In [ ]:
for epoch in range(100000):
    train(epoch+start_epoch)

Train Epoch: 0 iloader:0 Loss:5.557866: , (3.411 sec/step)
write finish
Train Epoch: 0 iloader:1 Loss:5.407401: , (3.346 sec/step)
Train Epoch: 0 iloader:2 Loss:5.301368: , (3.345 sec/step)
Train Epoch: 0 iloader:3 Loss:4.963304: , (3.344 sec/step)
Train Epoch: 0 iloader:4 Loss:5.461830: , (3.345 sec/step)
Train Epoch: 1 iloader:0 Loss:5.062638: , (3.344 sec/step)
write finish
Train Epoch: 1 iloader:1 Loss:5.047495: , (3.352 sec/step)
Train Epoch: 1 iloader:2 Loss:5.087583: , (3.350 sec/step)
Train Epoch: 1 iloader:3 Loss:5.653695: , (3.351 sec/step)
Train Epoch: 1 iloader:4 Loss:5.149059: , (3.345 sec/step)
Train Epoch: 2 iloader:0 Loss:5.045873: , (3.364 sec/step)
write finish
Train Epoch: 2 iloader:1 Loss:4.905088: , (3.368 sec/step)
Train Epoch: 2 iloader:2 Loss:4.744982: , (3.371 sec/step)
Train Epoch: 2 iloader:3 Loss:5.067408: , (3.367 sec/step)
Train Epoch: 2 iloader:4 Loss:5.778376: , (3.372 sec/step)
Train Epoch: 3 iloader:0 Loss:5.134910: , (3.377 sec/step)
write finish
Trai

Train Epoch: 26 iloader:2 Loss:5.171208: , (3.355 sec/step)
Train Epoch: 26 iloader:3 Loss:4.754235: , (3.350 sec/step)
Train Epoch: 26 iloader:4 Loss:4.957862: , (3.359 sec/step)
Train Epoch: 27 iloader:0 Loss:4.865154: , (3.376 sec/step)
write finish
Train Epoch: 27 iloader:1 Loss:4.794991: , (3.361 sec/step)
Train Epoch: 27 iloader:2 Loss:5.227072: , (3.356 sec/step)
Train Epoch: 27 iloader:3 Loss:4.709082: , (3.355 sec/step)
Train Epoch: 27 iloader:4 Loss:4.576166: , (3.352 sec/step)
Train Epoch: 28 iloader:0 Loss:4.894642: , (3.353 sec/step)
write finish
Train Epoch: 28 iloader:1 Loss:4.928898: , (3.357 sec/step)
Train Epoch: 28 iloader:2 Loss:5.192333: , (3.351 sec/step)
Train Epoch: 28 iloader:3 Loss:4.745818: , (3.353 sec/step)
Train Epoch: 28 iloader:4 Loss:4.853466: , (3.353 sec/step)
Train Epoch: 29 iloader:0 Loss:4.835082: , (3.370 sec/step)
write finish
Train Epoch: 29 iloader:1 Loss:4.734999: , (3.350 sec/step)
Train Epoch: 29 iloader:2 Loss:4.678761: , (3.355 sec/step)
T

Train Epoch: 52 iloader:3 Loss:4.616348: , (3.376 sec/step)
Train Epoch: 52 iloader:4 Loss:4.907684: , (3.382 sec/step)
Train Epoch: 53 iloader:0 Loss:4.535191: , (3.399 sec/step)
write finish
Train Epoch: 53 iloader:1 Loss:4.975483: , (3.388 sec/step)
Train Epoch: 53 iloader:2 Loss:4.743604: , (3.378 sec/step)
Train Epoch: 53 iloader:3 Loss:4.705423: , (3.377 sec/step)
Train Epoch: 53 iloader:4 Loss:4.845868: , (3.363 sec/step)
Train Epoch: 54 iloader:0 Loss:3.208977: , (3.353 sec/step)
write finish
Train Epoch: 54 iloader:1 Loss:4.598856: , (3.361 sec/step)
Train Epoch: 54 iloader:2 Loss:4.568089: , (3.353 sec/step)
Train Epoch: 54 iloader:3 Loss:4.959657: , (3.357 sec/step)
Train Epoch: 54 iloader:4 Loss:4.705436: , (3.353 sec/step)
Train Epoch: 55 iloader:0 Loss:4.510989: , (3.366 sec/step)
write finish
Train Epoch: 55 iloader:1 Loss:3.261402: , (3.357 sec/step)
Train Epoch: 55 iloader:2 Loss:4.706635: , (3.354 sec/step)
Train Epoch: 55 iloader:3 Loss:4.934013: , (3.357 sec/step)
T

Train Epoch: 78 iloader:4 Loss:4.982837: , (3.382 sec/step)
Train Epoch: 79 iloader:0 Loss:4.803292: , (3.390 sec/step)
write finish
Train Epoch: 79 iloader:1 Loss:4.466008: , (3.383 sec/step)
Train Epoch: 79 iloader:2 Loss:4.799454: , (3.380 sec/step)
Train Epoch: 79 iloader:3 Loss:3.783533: , (3.376 sec/step)
Train Epoch: 79 iloader:4 Loss:4.772820: , (3.369 sec/step)
Train Epoch: 80 iloader:0 Loss:4.826638: , (3.379 sec/step)
write finish
Train Epoch: 80 iloader:1 Loss:4.903940: , (3.379 sec/step)
Train Epoch: 80 iloader:2 Loss:3.554617: , (3.376 sec/step)
Train Epoch: 80 iloader:3 Loss:4.962130: , (3.378 sec/step)
Train Epoch: 80 iloader:4 Loss:4.879841: , (3.375 sec/step)
Train Epoch: 81 iloader:0 Loss:4.666832: , (3.365 sec/step)
write finish
Train Epoch: 81 iloader:1 Loss:4.709671: , (3.375 sec/step)
Train Epoch: 81 iloader:2 Loss:4.766846: , (3.376 sec/step)
Train Epoch: 81 iloader:3 Loss:4.585537: , (3.374 sec/step)
Train Epoch: 81 iloader:4 Loss:4.740732: , (3.379 sec/step)
T

Train Epoch: 105 iloader:0 Loss:4.585058: , (3.364 sec/step)
write finish
Train Epoch: 105 iloader:1 Loss:4.526569: , (3.368 sec/step)
Train Epoch: 105 iloader:2 Loss:4.902738: , (3.364 sec/step)
Train Epoch: 105 iloader:3 Loss:5.623612: , (3.368 sec/step)
Train Epoch: 105 iloader:4 Loss:4.644127: , (3.358 sec/step)
Train Epoch: 106 iloader:0 Loss:4.510067: , (3.385 sec/step)
write finish
Train Epoch: 106 iloader:1 Loss:4.582514: , (3.378 sec/step)
Train Epoch: 106 iloader:2 Loss:4.729449: , (3.365 sec/step)
Train Epoch: 106 iloader:3 Loss:4.755068: , (3.363 sec/step)
Train Epoch: 106 iloader:4 Loss:4.686248: , (3.362 sec/step)
Train Epoch: 107 iloader:0 Loss:4.465150: , (3.387 sec/step)
write finish
Train Epoch: 107 iloader:1 Loss:4.500347: , (3.359 sec/step)
Train Epoch: 107 iloader:2 Loss:4.644814: , (3.371 sec/step)
Train Epoch: 107 iloader:3 Loss:4.714014: , (3.359 sec/step)
Train Epoch: 107 iloader:4 Loss:5.296492: , (3.364 sec/step)
Train Epoch: 108 iloader:0 Loss:4.433876: , (3

Train Epoch: 130 iloader:4 Loss:4.610128: , (3.398 sec/step)
Train Epoch: 131 iloader:0 Loss:4.151983: , (3.366 sec/step)
write finish
Train Epoch: 131 iloader:1 Loss:4.645586: , (3.380 sec/step)
Train Epoch: 131 iloader:2 Loss:4.630166: , (3.372 sec/step)
Train Epoch: 131 iloader:3 Loss:4.337899: , (3.380 sec/step)
Train Epoch: 131 iloader:4 Loss:4.689037: , (3.372 sec/step)
Train Epoch: 132 iloader:0 Loss:4.661103: , (3.383 sec/step)
write finish
Train Epoch: 132 iloader:1 Loss:4.471372: , (3.388 sec/step)
Train Epoch: 132 iloader:2 Loss:5.176295: , (3.374 sec/step)
Train Epoch: 132 iloader:3 Loss:4.742886: , (3.377 sec/step)
Train Epoch: 132 iloader:4 Loss:5.352309: , (3.374 sec/step)
Train Epoch: 133 iloader:0 Loss:4.650817: , (3.401 sec/step)
write finish
Train Epoch: 133 iloader:1 Loss:4.773270: , (3.374 sec/step)
Train Epoch: 133 iloader:2 Loss:4.776898: , (3.378 sec/step)
Train Epoch: 133 iloader:3 Loss:4.873064: , (3.374 sec/step)
Train Epoch: 133 iloader:4 Loss:4.694152: , (3